In [1]:
# Import librarires
import pandas as pd
import numpy as np
import os
import seaborn as sns; sns.set(style="white", color_codes=True, rc={'figure.figsize':(11.7,8.27)})
import matplotlib.pyplot as plt
import re

# Create anndata and scanpy
import anndata as ad
import scanpy as sc
import scanpy.external as sce
import bbknn

# Import custom functions
import wget

In [3]:
# Set working directory
# Laptop
WD = "/Users/aj/Dropbox (Partners HealthCare)/Data/Vignesh_Lymphoma_tma/20190214_Datadump/"
# HMS
#WD = "C:/Users/ajn16/Dropbox (Partners HealthCare)/Data/Vignesh_Lymphoma_tma/20190214_Datadump/"
os.chdir(WD)

In [23]:
# Load Data
adata = sc.read("atcl.h5ad")

In [4]:
# Create Dataset
data = pd.read_csv('PTCLCycle1to9_CytMedian_nonorm.csv', delimiter=',', index_col=0)
meta = pd.read_csv('meta_new.csv', delimiter=',',index_col=0)
# Drop DAPI
data = data.drop(list(data.filter(regex='DAPI|BACK', axis=1)), axis=1)
# Convert to AnnData 
adata = sc.AnnData(data)
adata.obs = meta
# Subset Angioimmunoblatic T cell lymphoma
adata = adata[adata.obs['Disease'].isin(['Angioimmunoblastic Cell Lymphoma'])]
# Covert patient column into string 
adata.obs['patient'] = adata.obs['patient'].astype(str)
adata.obs['core'] = adata.obs['core'].astype(str)
# Remove markers after cycle 10
adata = adata[:,0:29]
# Remove other identified bad markers
columns = ['CD10', 'CD11B', 'CD57', 'FOXP3']
remove = np.in1d(adata.var_names, columns)
adata = adata[:, ~remove]
# Basic Filtering
sc.pp.filter_cells(adata, min_genes=25)
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1)

In [34]:
adata.X

array([[2.017e+03, 5.371e+03, 9.670e+02, ..., 1.620e+02, 6.100e+01,
        5.700e+01],
       [5.110e+02, 2.751e+03, 3.700e+02, ..., 8.600e+01, 1.900e+01,
        3.100e+01],
       [8.700e+01, 8.980e+02, 3.600e+01, ..., 1.530e+02, 5.000e+00,
        1.800e+01],
       ...,
       [1.580e+02, 4.410e+02, 5.190e+02, ..., 6.180e+02, 1.900e+01,
        7.000e+00],
       [1.300e+01, 1.030e+02, 1.800e+01, ..., 2.000e+00, 2.000e+00,
        6.000e+00],
       [6.900e+01, 5.310e+02, 2.180e+02, ..., 7.900e+02, 2.000e+01,
        8.000e+00]], dtype=float32)

In [29]:
adata

AnnData object with n_obs × n_vars = 55924 × 25 
    obs: 'core', 'Disease', 'StudyID', 'patient', 'n_genes'

In [33]:
# Write Data
adata.write("atcl.h5ad")